In [ ]:
%matplotlib notebook
import numpy as np
from numba import jit
from potentials import *
from distances import *
from sampling import *
from vv import vvJ
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def pot_gradient(coord, pbc=False,boxsize=(0,10), sig=1, eps=1):
    vecs = vectors(coord, boxsize, pbc)
    return gradients.LJ(vecs, sig, eps) + gradients.harmonic(coord,boxsize, pbc, r0 =(5, 5), k=5)
def pot(coord, pbc=False,boxsize=(0,10), sig=1, eps=1):
    vecs = vectors(coord, boxsize, pbc)
    dist = distances(vecs)
    return potentials.LJ(dist, eps, sig) + potentials.harmonic(coord, boxsize, pbc, r0 =(5, 5),k=5)

In [ ]:
r_init = mcmc(pot,50, 2, 10000, stepsize=0.01, beta=100, boxsize=(0,10), pbc=False)
v_init = np.zeros_like(r_init)
v_init[0] = [10,0]
r_matrix, v_matrix, a_matrix=vvJ(pot_gradient, r_init, v_init, 1, 10, 0.001)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
colors = np.arange(len(r_init))
scat = ax.scatter(r_init[:,0], r_init[:,1], c=colors)
circles = [plt.Circle(r, radius=0.5, fill=False) for i,r in enumerate(r_init)]
for c in circles:
    plt.gca().add_patch(c)
qax = ax.quiver(r_matrix[0,:,0], r_matrix[0,:,1], v_matrix[1,:,0], v_matrix[1,:,1],np.arange(len(r_init)),scale=50, width=0.005)
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
def animate(i):
    index = 4*i
    data = r_matrix[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    qax.set_UVC(v_matrix[index,:,0],v_matrix[index,:,1])
    qax.set_offsets(data)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=20, frames=750, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)